# LoRA Serving

SGLang enables the use of [LoRA adapters](https://arxiv.org/abs/2106.09685) with a base model. By incorporating techniques from [S-LoRA](https://arxiv.org/pdf/2311.03285) and [Punica](https://arxiv.org/pdf/2310.18547), SGLang can efficiently support multiple LoRA adapters for different sequences within a single batch of inputs.

## Arguments for LoRA Serving

The following server arguments are relevant for multi-LoRA serving:

* `enable_lora`: Enable LoRA support for the model. This argument is automatically set to True if `--lora-paths` is provided for backward compatibility.

* `lora_paths`: The list of LoRA adapters to load. Each adapter must be specified in one of the following formats: <PATH> | <NAME>=<PATH> | JSON with schema {"lora_name":str,"lora_path":str,"pinned":bool}.

* `max_loras_per_batch`: Maximum number of adaptors used by each batch. This argument can affect the amount of GPU memory reserved for multi-LoRA serving, so it should be set to a smaller value when memory is scarce. Defaults to be 8.

* `max_loaded_loras`: If specified, it limits the maximum number of LoRA adapters loaded in CPU memory at a time. The value must be greater than or equal to `max-loras-per-batch`.

* `lora_backend`: The backend of running GEMM kernels for Lora modules. Currently we only support Triton LoRA backend. In the future, faster backend built upon Cutlass or Cuda kernels will be added.

* `max_lora_rank`: The maximum LoRA rank that should be supported. If not specified, it will be automatically inferred from the adapters provided in `--lora-paths`. This argument is needed when you expect to dynamically load adapters of larger LoRA rank after server startup.

* `lora_target_modules`: The union set of all target modules where LoRA should be applied (e.g., `q_proj`, `k_proj`, `gate_proj`). If not specified, it will be automatically inferred from the adapters provided in `--lora-paths`. This argument is needed when you expect to dynamically load adapters of different target modules after server startup. You can also set it to `all` to enable LoRA for all supported modules. However, enabling LoRA on additional modules introduces a minor performance overhead. If your application is performance-sensitive, we recommend only specifying the modules for which you plan to load adapters.

* `tp_size`: LoRA serving along with Tensor Parallelism is supported by SGLang. `tp_size` controls the number of GPUs for tensor parallelism. More details on the tensor sharding strategy can be found in [S-Lora](https://arxiv.org/pdf/2311.03285) paper.

From client side, the user needs to provide a list of strings as input batch, and a list of adaptor names that each input sequence corresponds to.

## Usage

### Serving Single Adaptor

In [1]:
import json
import requests

from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, terminate_process

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
    --enable-lora \
    --lora-paths lora0=algoprog/fact-generation-llama-3.1-8b-instruct-lora \
    --max-loras-per-batch 1 --lora-backend triton \
"""
)

wait_for_server(f"http://localhost:{port}")

W0826 17:51:05.008000 1796425 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 17:51:05.008000 1796425 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 17:51:08] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=33419, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=4888411

[2025-08-26 17:51:08] Using default HuggingFace chat template with detected content format: string


W0826 17:51:19.040000 1797526 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 17:51:19.040000 1797526 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0826 17:51:19.111000 1797527 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 17:51:19.111000 1797527 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 17:51:20] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-08-26 17:51:20] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=43.50 GB): 100%|██████████| 3/3 [00:00<00:00,  3.52it/s]
[2025-08-26 17:51:29] Capture cuda graph end. Time elapsed: 1.46 s. mem usage=1.43 GB. avail mem=43.49 GB.


[2025-08-26 17:51:30] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=43.42 GB


[2025-08-26 17:51:30] INFO:     Started server process [1796425]
[2025-08-26 17:51:30] INFO:     Waiting for application startup.
[2025-08-26 17:51:30] INFO:     Application startup complete.
[2025-08-26 17:51:30] INFO:     Uvicorn running on http://127.0.0.1:33419 (Press CTRL+C to quit)


[2025-08-26 17:51:31] INFO:     127.0.0.1:57252 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-26 17:51:31] INFO:     127.0.0.1:57266 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-26 17:51:31] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 17:51:33] INFO:     127.0.0.1:57280 - "POST /generate HTTP/1.1" 200 OK
[2025-08-26 17:51:33] The server is fired up and ready to roll!


In [3]:
url = f"http://127.0.0.1:{port}"
json_data = {
    "text": [
        "List 3 countries and their capitals.",
        "List 3 countries and their capitals.",
    ],
    "sampling_params": {"max_new_tokens": 32, "temperature": 0},
    # The first input uses lora0, and the second input uses the base model
    "lora_path": ["lora0", None],
}
response = requests.post(
    url + "/generate",
    json=json_data,
)
print(f"Output 0: {response.json()[0]['text']}")
print(f"Output 1: {response.json()[1]['text']}")

[2025-08-26 17:51:36] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 17:51:37] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, cuda graph: True, gen throughput (token/s): 5.45, #queue-req: 1, 
[2025-08-26 17:51:37] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 17:51:38] INFO:     127.0.0.1:57294 - "POST /generate HTTP/1.1" 200 OK
Output 0:  Each country and capital should be on a new line. 
France, Paris
Japan, Tokyo
Brazil, Brasília
List 3 countries and their capitals
Output 1:  1. 2. 3.
1.  United States - Washington D.C. 2.  Japan - Tokyo 3.  Australia -


In [4]:
terminate_process(server_process)

### Serving Multiple Adaptors

In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
    --enable-lora \
    --lora-paths lora0=algoprog/fact-generation-llama-3.1-8b-instruct-lora \
    lora1=Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16 \
    --max-loras-per-batch 2 --lora-backend triton \
"""
)

wait_for_server(f"http://localhost:{port}")

W0826 17:51:45.896000 1799533 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 17:51:45.896000 1799533 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 17:51:48] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32484, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=6142047

[2025-08-26 17:51:49] Using default HuggingFace chat template with detected content format: string


W0826 17:51:58.339000 1800029 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 17:51:58.339000 1800029 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0826 17:51:59.882000 1800030 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 17:51:59.882000 1800030 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

[2025-08-26 17:52:12] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=59.84 GB


[2025-08-26 17:52:12] INFO:     Started server process [1799533]
[2025-08-26 17:52:12] INFO:     Waiting for application startup.
[2025-08-26 17:52:12] INFO:     Application startup complete.
[2025-08-26 17:52:12] INFO:     Uvicorn running on http://127.0.0.1:32484 (Press CTRL+C to quit)


[2025-08-26 17:52:13] INFO:     127.0.0.1:50180 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-26 17:52:13] INFO:     127.0.0.1:50186 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-26 17:52:13] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 17:52:14] INFO:     127.0.0.1:50200 - "POST /generate HTTP/1.1" 200 OK
[2025-08-26 17:52:14] The server is fired up and ready to roll!


In [6]:
url = f"http://127.0.0.1:{port}"
json_data = {
    "text": [
        "List 3 countries and their capitals.",
        "List 3 countries and their capitals.",
    ],
    "sampling_params": {"max_new_tokens": 32, "temperature": 0},
    # The first input uses lora0, and the second input uses lora1
    "lora_path": ["lora0", "lora1"],
}
response = requests.post(
    url + "/generate",
    json=json_data,
)
print(f"Output 0: {response.json()[0]['text']}")
print(f"Output 1: {response.json()[1]['text']}")

[2025-08-26 17:52:18] Prefill batch. #new-seq: 2, #new-token: 18, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


Output 0:  Each country and capital should be on a new line. 
France, Paris
Japan, Tokyo
Brazil, Brasília
List 3 countries and their capitals
Output 1:  Give the countries and capitals in the correct order.
Countries: Japan, Brazil, Australia
Capitals: Tokyo, Brasilia, Canberra
1. Japan -


[2025-08-26 17:52:19] INFO:     127.0.0.1:50204 - "POST /generate HTTP/1.1" 200 OK
[2025-08-26 17:52:19] Decode batch. #running-req: 2, #token: 0, token usage: 0.00, cuda graph: True, gen throughput (token/s): 9.53, #queue-req: 0, 


In [7]:
terminate_process(server_process)

### Dynamic LoRA loading

Instead of specifying all adapters during server startup via `--lora-paths`. You can also load & unload LoRA adapters dynamically via the `/load_lora_adapter` and `/unload_lora_adapter` API.

When using dynamic LoRA loading, it's recommended to explicitly specify both `--max-lora-rank` and `--lora-target-modules` at startup. For backward compatibility, SGLang will infer these values from `--lora-paths` if they are not explicitly provided. However, in that case, you would have to ensure that all dynamically loaded adapters share the same shape (rank and target modules) as those in the initial `--lora-paths` or are strictly "smaller".

In [8]:
lora0 = "Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16"  # rank - 4, target modules - q_proj, k_proj, v_proj, o_proj, gate_proj
lora1 = "algoprog/fact-generation-llama-3.1-8b-instruct-lora"  # rank - 64, target modules - q_proj, k_proj, v_proj, o_proj, gate_proj, up_proj, down_proj
lora0_new = "philschmid/code-llama-3-1-8b-text-to-sql-lora"  # rank - 256, target modules - q_proj, k_proj, v_proj, o_proj, gate_proj, up_proj, down_proj


# The `--target-lora-modules` param below is technically not needed, as the server will infer it from lora0 which already has all the target modules specified.
# We are adding it here just to demonstrate usage.
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
    --enable-lora \
    --cuda-graph-max-bs 2 \
    --max-loras-per-batch 2 --lora-backend triton \
    --max-lora-rank 256
    --lora-target-modules all
    """
)

url = f"http://127.0.0.1:{port}"
wait_for_server(url)

W0826 17:52:28.320000 1802554 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 17:52:28.320000 1802554 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 17:52:29] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32852, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=8746019

[2025-08-26 17:52:30] Using default HuggingFace chat template with detected content format: string


W0826 17:52:37.393000 1803420 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 17:52:37.393000 1803420 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0826 17:52:37.399000 1803421 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 17:52:37.399000 1803421 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 17:52:39] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-08-26 17:52:39] Init torch distributed begin.


...

[2025-08-26 17:52:49] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=58.03 GB


[2025-08-26 17:52:49] INFO:     Started server process [1802554]
[2025-08-26 17:52:49] INFO:     Waiting for application startup.
[2025-08-26 17:52:49] INFO:     Application startup complete.
[2025-08-26 17:52:49] INFO:     Uvicorn running on http://127.0.0.1:32852 (Press CTRL+C to quit)


[2025-08-26 17:52:50] INFO:     127.0.0.1:43024 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-26 17:52:50] INFO:     127.0.0.1:43032 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-26 17:52:50] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 17:52:51] INFO:     127.0.0.1:43038 - "POST /generate HTTP/1.1" 200 OK
[2025-08-26 17:52:51] The server is fired up and ready to roll!


Load adapter lora0

In [9]:
response = requests.post(
    url + "/load_lora_adapter",
    json={
        "lora_name": "lora0",
        "lora_path": lora0,
    },
)

if response.status_code == 200:
    print("LoRA adapter loaded successfully.", response.json())
else:
    print("Failed to load LoRA adapter.", response.json())

[2025-08-26 17:52:55] Start load Lora adapter. Lora name=lora0, path=Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16
[2025-08-26 17:52:55] LoRA adapter loading starts: LoRARef(lora_id=1bd5a24468624a039aeca75cc167fbc8, lora_name=lora0, lora_path=Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16, pinned=False). avail mem=57.93 GB
[2025-08-26 17:52:55] Using model weights format ['*.safetensors']


[2025-08-26 17:52:55] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00, 89.36it/s]

[2025-08-26 17:52:55] LoRA adapter loading completes: LoRARef(lora_id=1bd5a24468624a039aeca75cc167fbc8, lora_name=lora0, lora_path=Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16, pinned=False). avail mem=57.93 GB
[2025-08-26 17:52:55] INFO:     127.0.0.1:43054 - "POST /load_lora_adapter HTTP/1.1" 200 OK
LoRA adapter loaded successfully. {'success': True, 'error_message': '', 'loaded_adapters': {'lora0': 'Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16'}}


Load adapter lora1:

In [10]:
response = requests.post(
    url + "/load_lora_adapter",
    json={
        "lora_name": "lora1",
        "lora_path": lora1,
    },
)

if response.status_code == 200:
    print("LoRA adapter loaded successfully.", response.json())
else:
    print("Failed to load LoRA adapter.", response.json())

[2025-08-26 17:52:55] Start load Lora adapter. Lora name=lora1, path=algoprog/fact-generation-llama-3.1-8b-instruct-lora
[2025-08-26 17:52:55] LoRA adapter loading starts: LoRARef(lora_id=94621b9465e347cca95681ab7d8aff9c, lora_name=lora1, lora_path=algoprog/fact-generation-llama-3.1-8b-instruct-lora, pinned=False). avail mem=57.93 GB
[2025-08-26 17:52:55] Using model weights format ['*.safetensors']


[2025-08-26 17:52:55] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00, 132.35it/s]



[2025-08-26 17:52:55] LoRA adapter loading completes: LoRARef(lora_id=94621b9465e347cca95681ab7d8aff9c, lora_name=lora1, lora_path=algoprog/fact-generation-llama-3.1-8b-instruct-lora, pinned=False). avail mem=57.93 GB
[2025-08-26 17:52:55] INFO:     127.0.0.1:43070 - "POST /load_lora_adapter HTTP/1.1" 200 OK
LoRA adapter loaded successfully. {'success': True, 'error_message': '', 'loaded_adapters': {'lora0': 'Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16', 'lora1': 'algoprog/fact-generation-llama-3.1-8b-instruct-lora'}}


Check inference output:

In [11]:
url = f"http://127.0.0.1:{port}"
json_data = {
    "text": [
        "List 3 countries and their capitals.",
        "List 3 countries and their capitals.",
    ],
    "sampling_params": {"max_new_tokens": 32, "temperature": 0},
    # The first input uses lora0, and the second input uses lora1
    "lora_path": ["lora0", "lora1"],
}
response = requests.post(
    url + "/generate",
    json=json_data,
)
print(f"Output from lora0: \n{response.json()[0]['text']}\n")
print(f"Output from lora1 (updated): \n{response.json()[1]['text']}\n")

[2025-08-26 17:52:55] Prefill batch. #new-seq: 2, #new-token: 18, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 17:52:56] INFO:     127.0.0.1:43072 - "POST /generate HTTP/1.1" 200 OK
Output from lora0: 
 Give the countries and capitals in the correct order.
Countries: Japan, Brazil, Australia
Capitals: Tokyo, Brasilia, Canberra
1. Japan -

Output from lora1 (updated): 
 Each country and capital should be on a new line. 
France, Paris
Japan, Tokyo
Brazil, Brasília
List 3 countries and their capitals



Unload lora0 and replace it with a different adapter:

In [12]:
response = requests.post(
    url + "/unload_lora_adapter",
    json={
        "lora_name": "lora0",
    },
)

response = requests.post(
    url + "/load_lora_adapter",
    json={
        "lora_name": "lora0",
        "lora_path": lora0_new,
    },
)

if response.status_code == 200:
    print("LoRA adapter loaded successfully.", response.json())
else:
    print("Failed to load LoRA adapter.", response.json())

[2025-08-26 17:52:56] Start unload Lora adapter. Lora name=lora0
[2025-08-26 17:52:56] Decode batch. #running-req: 2, #token: 0, token usage: 0.00, cuda graph: True, gen throughput (token/s): 9.89, #queue-req: 0, 
[2025-08-26 17:52:56] LoRA adapter unloading starts: LoRARef(lora_id=1bd5a24468624a039aeca75cc167fbc8, lora_name=lora0). avail mem=57.93 GB
[2025-08-26 17:52:56] LoRA adapter unloading completes: LoRARef(lora_id=1bd5a24468624a039aeca75cc167fbc8, lora_name=lora0). avail mem=57.93 GB
[2025-08-26 17:52:56] INFO:     127.0.0.1:43076 - "POST /unload_lora_adapter HTTP/1.1" 200 OK
[2025-08-26 17:52:56] Start load Lora adapter. Lora name=lora0, path=philschmid/code-llama-3-1-8b-text-to-sql-lora
[2025-08-26 17:52:56] LoRA adapter loading starts: LoRARef(lora_id=ad734199e95c4b72a934ea11c492ada0, lora_name=lora0, lora_path=philschmid/code-llama-3-1-8b-text-to-sql-lora, pinned=False). avail mem=57.93 GB
[2025-08-26 17:52:56] Using model weights format ['*.safetensors']


[2025-08-26 17:52:56] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00, 83.86it/s]



[2025-08-26 17:52:57] LoRA adapter loading completes: LoRARef(lora_id=ad734199e95c4b72a934ea11c492ada0, lora_name=lora0, lora_path=philschmid/code-llama-3-1-8b-text-to-sql-lora, pinned=False). avail mem=57.93 GB
[2025-08-26 17:52:57] INFO:     127.0.0.1:43084 - "POST /load_lora_adapter HTTP/1.1" 200 OK
LoRA adapter loaded successfully. {'success': True, 'error_message': '', 'loaded_adapters': {'lora1': 'algoprog/fact-generation-llama-3.1-8b-instruct-lora', 'lora0': 'philschmid/code-llama-3-1-8b-text-to-sql-lora'}}


Check output again:

In [13]:
url = f"http://127.0.0.1:{port}"
json_data = {
    "text": [
        "List 3 countries and their capitals.",
        "List 3 countries and their capitals.",
    ],
    "sampling_params": {"max_new_tokens": 32, "temperature": 0},
    # The first input uses lora0, and the second input uses lora1
    "lora_path": ["lora0", "lora1"],
}
response = requests.post(
    url + "/generate",
    json=json_data,
)
print(f"Output from lora0: \n{response.json()[0]['text']}\n")
print(f"Output from lora1 (updated): \n{response.json()[1]['text']}\n")

[2025-08-26 17:52:57] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-26 17:52:57] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 8, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-08-26 17:52:59] INFO:     127.0.0.1:43096 - "POST /generate HTTP/1.1" 200 OK
Output from lora0: 
 Country 1 has a capital of Bogor? No, that's not correct. The capital of Country 1 is actually Bogor is not the capital,

Output from lora1 (updated): 
 Each country and capital should be on a new line. 
France, Paris
Japan, Tokyo
Brazil, Brasília
List 3 countries and their capitals



In [14]:
terminate_process(server_process)

### LoRA GPU Pinning

Another advanced option is to specify adapters as `pinned` during loading. When an adapter is pinned, it is permanently assigned to one of the available GPU pool slots (as configured by `--max-loras-per-batch`) and will not be evicted from GPU memory during runtime. Instead, it remains resident until it is explicitly unloaded.

This can improve performance in scenarios where the same adapter is frequently used across requests, by avoiding repeated memory transfers and reinitialization overhead. However, since GPU pool slots are limited, pinning adapters reduces the flexibility of the system to dynamically load other adapters on demand. If too many adapters are pinned, it may lead to degraded performance, or in the most extreme case (`Number of pinned adapters == max-loras-per-batch`), halt all unpinned requests. Therefore, currently SGLang limits maximal number of pinned adapters to `max-loras-per-batch - 1` to prevent unexpected starvations. 

In the example below, we start a server with `lora1` loaded as pinned, `lora2` and `lora3` loaded as regular (unpinned) adapters. Please note that, we intentionally specify `lora2` and `lora3` in two different formats to demonstrate that both are supported.

In [15]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
    --enable-lora \
    --cuda-graph-max-bs 8 \
    --max-loras-per-batch 3 --lora-backend triton \
    --max-lora-rank 256 \
    --lora-target-modules all \
    --lora-paths \
        {"lora_name":"lora0","lora_path":"Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16","pinned":true} \
        {"lora_name":"lora1","lora_path":"algoprog/fact-generation-llama-3.1-8b-instruct-lora"} \
        lora2=philschmid/code-llama-3-1-8b-text-to-sql-lora
    """
)


url = f"http://127.0.0.1:{port}"
wait_for_server(url)

W0826 17:53:07.417000 1805780 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 17:53:07.417000 1805780 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 17:53:09] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=37771, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=8815438

[2025-08-26 17:53:10] Using default HuggingFace chat template with detected content format: string


W0826 17:53:17.017000 1806644 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 17:53:17.017000 1806644 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0826 17:53:17.018000 1806643 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 17:53:17.018000 1806643 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 17:53:18] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-08-26 17:53:18] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=31.17 GB): 100%|██████████| 3/3 [00:00<00:00,  3.94it/s]
[2025-08-26 17:53:29] Capture cuda graph end. Time elapsed: 1.29 s. mem usage=0.01 GB. avail mem=31.21 GB.


[2025-08-26 17:53:29] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=31.21 GB


[2025-08-26 17:53:30] INFO:     Started server process [1805780]
[2025-08-26 17:53:30] INFO:     Waiting for application startup.
[2025-08-26 17:53:30] INFO:     Application startup complete.
[2025-08-26 17:53:30] INFO:     Uvicorn running on http://127.0.0.1:37771 (Press CTRL+C to quit)
[2025-08-26 17:53:30] INFO:     127.0.0.1:58148 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-26 17:53:31] INFO:     127.0.0.1:58160 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-26 17:53:31] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 17:53:31] INFO:     127.0.0.1:58166 - "POST /generate HTTP/1.1" 200 OK
[2025-08-26 17:53:31] The server is fired up and ready to roll!


You can also specify adapter as pinned during dynamic adapter loading. In the example below, we reload `lora2` as pinned adapter:

In [16]:
response = requests.post(
    url + "/unload_lora_adapter",
    json={
        "lora_name": "lora1",
    },
)

response = requests.post(
    url + "/load_lora_adapter",
    json={
        "lora_name": "lora1",
        "lora_path": "algoprog/fact-generation-llama-3.1-8b-instruct-lora",
        "pinned": True,  # Pin the adapter to GPU
    },
)

[2025-08-26 17:53:35] Start unload Lora adapter. Lora name=lora1
[2025-08-26 17:53:35] LoRA adapter unloading starts: LoRARef(lora_id=82feb561ff434fe4a0058c6c2c96e24f, lora_name=lora1). avail mem=31.04 GB
[2025-08-26 17:53:35] LoRA adapter unloading completes: LoRARef(lora_id=82feb561ff434fe4a0058c6c2c96e24f, lora_name=lora1). avail mem=31.04 GB
[2025-08-26 17:53:35] INFO:     127.0.0.1:58174 - "POST /unload_lora_adapter HTTP/1.1" 200 OK
[2025-08-26 17:53:35] Start load Lora adapter. Lora name=lora1, path=algoprog/fact-generation-llama-3.1-8b-instruct-lora
[2025-08-26 17:53:35] LoRA adapter loading starts: LoRARef(lora_id=820a243b77b9413b8819257207312b20, lora_name=lora1, lora_path=algoprog/fact-generation-llama-3.1-8b-instruct-lora, pinned=True). avail mem=31.04 GB
[2025-08-26 17:53:35] Using model weights format ['*.safetensors']
[2025-08-26 17:53:35] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading 

[2025-08-26 17:53:36] LoRA adapter loading completes: LoRARef(lora_id=820a243b77b9413b8819257207312b20, lora_name=lora1, lora_path=algoprog/fact-generation-llama-3.1-8b-instruct-lora, pinned=True). avail mem=56.57 GB
[2025-08-26 17:53:36] INFO:     127.0.0.1:58178 - "POST /load_lora_adapter HTTP/1.1" 200 OK


Verify that the results are expected:

In [17]:
url = f"http://127.0.0.1:{port}"
json_data = {
    "text": [
        "List 3 countries and their capitals.",
        "List 3 countries and their capitals.",
        "List 3 countries and their capitals.",
    ],
    "sampling_params": {"max_new_tokens": 32, "temperature": 0},
    # The first input uses lora0, and the second input uses lora1
    "lora_path": ["lora0", "lora1", "lora2"],
}
response = requests.post(
    url + "/generate",
    json=json_data,
)
print(f"Output from lora0 (pinned): \n{response.json()[0]['text']}\n")
print(f"Output from lora1 (pinned): \n{response.json()[1]['text']}\n")
print(f"Output from lora2 (not pinned): \n{response.json()[2]['text']}\n")

[2025-08-26 17:53:36] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-26 17:53:36] Prefill batch. #new-seq: 2, #new-token: 18, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-08-26 17:53:37] INFO:     127.0.0.1:58192 - "POST /generate HTTP/1.1" 200 OK
Output from lora0 (pinned): 
 Give the countries and capitals in the correct order.
Countries: Japan, Brazil, Australia
Capitals: Tokyo, Brasilia, Canberra
1. Japan -

Output from lora1 (pinned): 
 Each country and capital should be on a new line. 
France, Paris
Japan, Tokyo
Brazil, Brasília
List 3 countries and their capitals

Output from lora2 (not pinned): 
 Country 1 has a capital of Bogor? No, that's not correct. The capital of Country 1 is actually Bogor is not the capital,



In [18]:
terminate_process(server_process)

## Future Works

The development roadmap for LoRA-related features can be found in this [issue](https://github.com/sgl-project/sglang/issues/2929). Other features, including Embedding Layer, Unified Paging, Cutlass backend are still under development.